In [1]:
#hide
#from nbdev import *

In [1]:
#hide
%reload_ext autoreload
%autoreload 2

In [2]:
#hide
from functools import partial
import torch

In [3]:
#hide
# from model_constructor.layers import *

# model_constructor

> Constructor to create pytorch model. 

## Install

`pip install model-constructor`

Or install from repo:

`pip install git+https://github.com/ayasyrev/model_constructor.git`

## How to use

First import constructor class, then create model constructor object.

Now you can change every part of model.

In [4]:
from model_constructor import ModelConstructor

In [5]:
mc = ModelConstructor()

In [6]:
mc

MC constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

Now we have model consructor, default setting as xresnet18. And we can get model after call it.

In [7]:
#collapse_output
model = mc()
model

Sequential(
  MC
  (stem): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): ResBlock(
        (convs): Sequential(
          (conv_0)

If you want to change model, just change constructor parameters.  
Lets create xresnet50.

In [8]:
mc.expansion = 4
mc.layers = [3,4,6,3]

Now we can look at model body and if we call constructor - we have pytorch model!

In [9]:
#collapse_output
mc.body

Sequential(
  (l_0): Sequential(
    (bl_0): ResBlock(
      (convs): Sequential(
        (conv_0): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvBnAct(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_2): ConvBnAct(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (id_conv): Sequential(
        (id_conv): ConvBnAct(
          (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(2

In [10]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

/home/jzz/anaconda3/envs/mc_dev/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([16, 1000])


In [11]:
#hide
mc.block_sizes

[16, 64, 128, 256, 512]

## More modification.

Main purpose of this module - fast and easy modify model.
And here is the link to more modification to beat Imagenette leaderboard with add MaxBlurPool and modification to ResBlock https://github.com/ayasyrev/imagenette_experiments/blob/master/ResnetTrick_create_model_fit.ipynb  

But now lets create model as mxresnet50 from fastai forums tread https://forums.fast.ai/t/how-we-beat-the-5-epoch-imagewoof-leaderboard-score-some-new-techniques-to-consider  


Lets create mxresnet constructor.

In [12]:
mc = ModelConstructor(name='MxResNet')

Then lets modify stem.

In [13]:
mc.stem_sizes = [3,32,64,64]

Now lets change activation function to Mish.
Here is link to forum disscussion https://forums.fast.ai/t/meet-mish-new-activation-function-possible-successor-to-relu  
We'v got Mish is in model_constructor.activations, but from pytorch 1.9 take it from torch:

In [14]:
# from model_constructor.activations import Mish
from torch.nn import Mish

In [15]:
mc.act_fn = Mish()

In [16]:
mc

MxResNet constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [17]:
#collapse_output
mc()

Sequential(
  MxResNet
  (stem): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (stem_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (l_0): Sequential(
      (bl_0): ResBlock(
        (convs): Sequential(
          (conv_0): ConvBnAct(
            (conv

### MXResNet50

Now lets make MxResNet50

In [18]:
mc.expansion = 4
mc.layers = [3,4,6,3]
mc.name = 'mxresnet50'

Now we have mxresnet50 constructor.  
We can inspect every parts of it.  
And after call it we got model.

In [19]:
mc

mxresnet50 constructor
  c_in: 3, c_out: 1000
  expansion: 4, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

In [20]:
#collapse_output
mc.stem.conv_1

ConvBnAct(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): Mish()
)

In [21]:
#collapse_output
mc.body.l_0.bl_0

ResBlock(
  (convs): Sequential(
    (conv_0): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_1): ConvBnAct(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): Mish()
    )
    (conv_2): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (id_conv): Sequential(
    (id_conv): ConvBnAct(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (act_fn): Mish()
)

In [22]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


We can get model direct way:

In [23]:
mc = ModelConstructor(name="MxResNet", act_fn=Mish(), layers=[3,4,6,3], expansion=4, stem_sizes=[32,64,64])
model = mc()

In [24]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])


## YaResNet

Now lets change Resblock to YaResBlock (Yet another ResNet, former NewResBlock) is in lib from version 0.1.0

In [25]:
from model_constructor.yaresnet import YaResBlock

In [26]:
mc.block = YaResBlock

That all. Now we have YaResNet constructor

In [27]:
#collapse_output
mc.name = 'YaResNet'
mc

YaResNet constructor
  c_in: 3, c_out: 1000
  expansion: 4, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 64, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [3, 4, 6, 3]

Let see what we have.

In [30]:
#collapse_output
mc.body.l_1.bl_0

In [31]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

## First version

First version, it deprecated, but still here for compatability.

In [27]:
from model_constructor.net import Net

In [28]:
mc = Net()

In [29]:
mc

Net constructor
  c_in: 3, c_out: 1000
  expansion: 1, groups: 1, dw: False, div_groups: None
  sa: False, se: False
  stem sizes: [3, 32, 32, 64], stride on 0
  body sizes [64, 128, 256, 512]
  layers: [2, 2, 2, 2]

In [30]:
#hide
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = mc()(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([16, 1000])
